## Determining Fantasy Baseball Draft Order

I feel like I've been victimized by the "randomness" of the previous processes to determine draft order, so instead of blaming my failures at winning fantasy baseball on my strategy, I choose to believe it's the process we use to generate the random numbers for determining draft pick order. This is an attempt to use a more robust form of generating random numbers that theoretically is also the most fair method. Most software implementations (Microsoft Excel, Google Sheets, base Python) use a method that appears random, but actually isn't.

### Different Types of Randomness
Computers are actually incapable of generating *truly* random numbers. They can often create something approaching random (*pseudo*-random), which is usually good enough for most purposes (probably even this one), but I'm the one complaining so I'll put in the work to implement a "more" random method: a [cryptographically secure pseudo-random number generator](https://en.wikipedia.org/wiki/Cryptographically_secure_pseudorandom_number_generator) (CSPRNG). A CSPRNG generates random numbers using a process that is not repeatable nor reversible, which is necessary for cryptography, and definitely overkill for our purposes.

### The Odds
The Championship winner (It Burns when I Pete) and runner-up (Honey Nut Ichiros) get 12th and 11th pick, respectively. 

The rest of the order is randomly assigned using the following chances of first pick:

Finish | Team | Odds
-|-|-
12th | Non-Fungible Torkelsons | 33.00 %
11th | Jellicle Cats | 22.00 %
10th | Calvin Klein Defy | 15.00 %
9th | Get Lowe | 10.00 %
8th | Giancarlo for Mayor | 6.50 %
7th | Donaldson's Man Bun | 5.50 %
6th | RBI'd for Pleasure | 3.50 %
5th | Monster Dongs | 2.50 %
4th | Jose Can You See | 1.50 %
3rd | Montana Wildhacks | 0.50 %

### Methodology

To reduce errors with small sample sizes, 1,000,000 balls will be entered into the draw. Each team will have a number of balls equal to their percentage, i.e. Non-Fungible Torkelsons receives 330,000 entries. The first ball gets the first pick, simple enough. The second ball drawn (and all subsequent balls), if it does not belong to a team already picked, chooses the next pick. Balls belonging to teams already chosen are technically put back into the pool, but it doesn't really affect anything with this methodology. This process continues until all teams are assigned draft positions.

Now the code:

In [1]:
# Import libraries
import pandas as pd
import secrets

# Setup dictionary with percentages, as above
percs = {'Non-Fungible Torkelsons': 330_000,
         'Jellicle Cats': 220_000,
         'Calvin Klein Defy': 150_000,
         'Get Lowe': 100_000,
         'Giancarlo for Mayor': 65_000,
         "Donaldson's Man Bun": 55_000,
         "RBI'd for Pleasure": 35_000,
         'Monster Dongs': 25_000,
         'Jose Can You See': 15_000,
         'Montana Wildhacks': 5_000}

# Create dataframe of balls with names
balls = pd.Series(dtype='str')
for i in percs:
    temp = pd.Series(i, index=range(percs[i]))
    balls = pd.concat([balls, temp], axis=0).reset_index(drop=True)

# Shuffle the dataframe for peace of mind
balls = balls.sample(frac=1, replace=False).reset_index(drop=True)

# Verify the counts
balls.value_counts()    

Non-Fungible Torkelsons    330000
Jellicle Cats              220000
Calvin Klein Defy          150000
Get Lowe                   100000
Giancarlo for Mayor         65000
Donaldson's Man Bun         55000
RBI'd for Pleasure          35000
Monster Dongs               25000
Jose Can You See            15000
Montana Wildhacks            5000
dtype: int64

## Let's Do It

Now without further ado, the part of the code that actually does the picking.

The steps are:
1. Create empty list
1. Randomly choose a ball from the 1 million balls (using the CSPRNG), and add that name to the list
1. Randomly choose another ball and check if the name is in the list already
    1. If the name is already in the list, skip
    1. If the name is *not* in the list, add to end of list
1. Pick again until list is full

We also can count how many balls were picked before the list was complete.

In [2]:
order = []
count = 0
while len(order) < 10:
    pick = secrets.choice(balls)
    if pick in order:
        pass
    else:
        order.append(pick)
    count += 1
display(order)
print('\n', f'After {count} balls')

['Jellicle Cats',
 'Non-Fungible Torkelsons',
 'Giancarlo for Mayor',
 'Get Lowe',
 'Calvin Klein Defy',
 "Donaldson's Man Bun",
 "RBI'd for Pleasure",
 'Monster Dongs',
 'Jose Can You See',
 'Montana Wildhacks']


 After 146 balls
